In [1]:
%display typeset

def SP(S, P, X):
    '''
    Sterographically project point X from S onto a plane defined by P where P is the point on the plane closest to S
    '''
    return S + ((P-S)*(P-S))/((X-S)*(P-S)) * (X-S)

S = vector([1,0,0,0,0,0])
P = vector([1/3,1/3,1/3,0,0,0])

def Sphere(u, v):
    return vector([cos(u)*cos(v), sin(u)*cos(v), sin(v)])

def Veronese(x, y, z):
    return vector([x^2, y^2, z^2, sqrt(2)*y*z, sqrt(2)*z*x, sqrt(2)*x*y])

basis = [vector(v).normalized() for v in [
    [-1,1,0,0,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,1]
]]

def project6to4(v):
    '''
    Project 6D vector v to 4D subspace of the hyperplane where Veronese surface is
    '''
    return vector([b*v for b in basis])

In [2]:
var('u,v')
Veronese(*Sphere(u, v)).subs(u=pi/2, v=pi/4)

(0, 1/2, 1/2, 1/2*sqrt(2), 0, 0)

In [3]:
def c(u, v):
    return ((u+pi)/2/pi).n()

p = parametric_plot3d([project6to4(Veronese(*Sphere(u,v)))[i] for i in [0,1,2]], (u,-pi,pi), (v,0,pi/2), \
                      plot_points=[80,40], color=(c, colormaps.hsv)) 
show(p, viewer='jmol')
p.save('test.png', viewer='jmol', frame=False, figsize=10, zoom=1.5)

Graphics3d Object

In [4]:
var('u,v')

icoVert = [vector(a).normalized() for a in icosahedron().vertices()]
icoVert2 = [vector(a).normalized() for a in [
    (0, 1, golden_ratio), (golden_ratio, 0, 1), (1, golden_ratio, 0),
    (0, -1, golden_ratio), (golden_ratio, 0, -1), (-1, golden_ratio, 0),
    (0, 1, -golden_ratio), (-golden_ratio, 0, 1), (1, -golden_ratio, 0),
    (0, -1, -golden_ratio), (-golden_ratio, 0, -1), (-1, -golden_ratio, 0),
]]

def projectVeroneseTo3DCart(X, coordinates):
    return vector([project6to4(Veronese(*X))[i] for i in coordinates])

def projectVeroneseTo3DSph(X, coordinates):
    return vector([project6to4(Veronese(*Sphere(*X)))[i] for i in coordinates])

def projectVeroneseTo3DRotate(X, angle):
    v = project6to4(Veronese(*X))
    return vector([v[0], v[1], cos(angle)*v[2]+sin(angle)*v[3]])

def projectVeroneseTo3DSphRotate(X, angle):
    v = project6to4(Veronese(*Sphere(*X)))
    return vector([v[0], v[1], cos(angle)*v[2]+sin(angle)*v[3]])

coorSelect = [0,2,3]


@interact
def interaction(angle=pi/4):
    V = [projectVeroneseTo3DRotate(X, angle) for X in icoVert[0:6]]
    Vp = [-projectVeroneseTo3DRotate(X, angle) for X in icoVert[0:6]]

    vert1 = point3d(V, size=20)
    vert2 = point3d(Vp, size=20, color='red')
    surface = parametric_plot3d(projectVeroneseTo3DSphRotate((u, v), angle), (u,-pi,pi), (v,0,pi/2), color='green', opacity=0.5)

    edges1 = line3d([[projectVeroneseTo3DRotate(X, angle) for X in icoVert[0:6]][i] for i in [0,1,2,0,3,1,3,2,4,0,4,1,4,3,5,0,5,1,5,2,5,3,5,4]], color='blue')
    edges2 = line3d([[-projectVeroneseTo3DRotate(X, angle) for X in icoVert[0:6]][i] for i in [0,1,2,0,3,1,3,2,4,0,4,1,4,3,5,0,5,1,5,2,5,3,5,4]], color='red')

    triangle1 = polygons3d([[3,4,5]], V, color='blue')
    

    show(vert1+vert2+edges1+edges2+triangle1, viewer='threejs')

SW50ZXJhY3RpdmUgZnVuY3Rpb24gPGZ1bmN0aW9uIGludGVyYWN0aW9uIGF0IDB4NmZlMDk5NDQzOTg+IHdpdGggMSB3aWRnZXQKICBhbmdsZTogRXZhbFRleHQodmFsdWU9dScxLzQqcGknLCDigKY=


In [5]:
# defined vertices and paths
V = [projectVeroneseTo3DCart(X, coorSelect) for X in icoVert[0:6]]
V_neg = [-projectVeroneseTo3DCart(X, coorSelect) for X in icoVert[0:6]]

V_mid = []
simplex_path = [0,1,2,0, 3,1,2,3, 4,0,1,4,2,3,4]

for v in V:
    V_mid.append([])
    for v_neg in V_neg:
        V_mid[-1].append((v+v_neg)/2)
        
V_mid_flattened = [a for a in flatten(V_mid) if a != vector([0,0,0])]

In [6]:
# show all edges

p = point3d(V_mid_flattened, size=20, color='yellow') + \
    sum([line3d([[a for a in V_mid[j] if a!=vector([0,0,0])][i] for i in simplex_path], color='black', radius=0.005)
            for j in range(6)]) + \
    sum([line3d([[v[j] for v in V_mid if v[j]!=vector([0,0,0])][i] for i in simplex_path], color='red', radius=0.005)
            for j in range(6)])
show(p, viewer='threejs', aspect_ratio=1)
p.save('cartan_graph.png', figsize=(12,20), dpi=300)
p.save('cartan_graph.stl')

Graphics3d Object

AttributeError: 'Graphics3dGroup' object has no attribute 'face_list'

In [ ]:
# show edges of V <-> vertices of -V

p2 = point3d(V_mid_flattened, size=20, color='yellow') + \
    sum([line3d([[a for a in V_mid[j] if a!=vector([0,0,0])][i] for i in simplex_path], color='black', radius=0.005)
            for j in range(6)])
show(p2, viewer='threejs', aspect_ratio=1)

In [ ]:
# show edges of -V <-> vertices of V

p3 = point3d(V_mid_flattened, size=20, color='yellow') + \
     sum([line3d([[v[j] for v in V_mid if v[j]!=vector([0,0,0])][i] for i in simplex_path], color='red', radius=0.005)
            for j in range(6)])
show(p3, viewer='threejs')

In [ ]:
# one prism

i1, i2, i3 = 1,)3,5
j1, j2 = 0,2

p4 = point3d(V_mid_flattened, size=40, color='yellow') + \
    line3d([V_mid[i1][j1], V_mid[i2][j1], V_mid[i3][j1], V_mid[i1][j1]], color='black', thickness=0.1) + \
    line3d([V_mid[i1][j2], V_mid[i2][j2], V_mid[i3][j2], V_mid[i1][j2]], color='black', thickness=0.1) + \
    line3d([V_mid[i1][j1], V_mid[i1][j2]], color='red', thickness=0.1) + \
    line3d([V_mid[i2][j1], V_mid[i2][j2]], color='red', thickness=0.1) + \
    line3d([V_mid[i3][j1], V_mid[i3][j2]], color='red', thickness=0.1)
    
show(p4)

In [ ]:
# one torus

i1, i2, i3 = 1,3,5
j1, j2, j3 = 0,2,4

p5 = point3d(V_mid_flattened, size=40, color='yellow') + \
     line3d([V_mid[i1][j1], V_mid[i2][j1], V_mid[i3][j1], V_mid[i1][j1]], color='black', thickness=0.1) + \
     line3d([V_mid[i1][j2], V_mid[i2][j2], V_mid[i3][j2], V_mid[i1][j2]], color='black', thickness=0.1) + \
     line3d([V_mid[i1][j3], V_mid[i2][j3], V_mid[i3][j3], V_mid[i1][j3]], color='black', thickness=0.1) + \
     \
     line3d([V_mid[i1][j1], V_mid[i1][j2]], color='red', thickness=0.1) + \
     line3d([V_mid[i2][j1], V_mid[i2][j2]], color='red', thickness=0.1) + \
     line3d([V_mid[i3][j1], V_mid[i3][j2]], color='red', thickness=0.1) + \
     \
     line3d([V_mid[i1][j2], V_mid[i1][j3]], color='red', thickness=0.1) + \
     line3d([V_mid[i2][j2], V_mid[i2][j3]], color='red', thickness=0.1) + \
     line3d([V_mid[i3][j2], V_mid[i3][j3]], color='red', thickness=0.1) + \
     \
     line3d([V_mid[i1][j3], V_mid[i1][j1]], color='red', thickness=0.1) + \
     line3d([V_mid[i2][j3], V_mid[i2][j1]], color='red', thickness=0.1) + \
     line3d([V_mid[i3][j3], V_mid[i3][j1]], color='red', thickness=0.1)
    
show(p5)